# Capacity and Targets

In [ ]:
import os
import sys

In [ ]:
# append path to find utils module in urbansim
cwd = os.getcwd() 
parentdir =  os.path.abspath(os.path.join(cwd, os.pardir))
parentdir2 =  os.path.abspath(os.path.join(parentdir, os.pardir))
sys.path.append(parentdir2) # to get path to utils module

In [ ]:
import pandas as pd
import numpy as np
import utils

In [ ]:
from sqlalchemy import create_engine
from database import get_connection_string

In [ ]:
%matplotlib inline

In [ ]:
# connect to database
db_connection_string = get_connection_string('..\..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

In [ ]:
# get versions
versions = utils.yaml_to_dict('../../data/scenario_config.yaml', 'scenario')

# Get Capacity

In [ ]:
capacity_sql = '''
SELECT jcpa,jcpa_name,COALESCE(sum(du),0) + sum(COALESCE(capacity_3,capacity_2)) as capacity
    FROM [urbansim].[urbansim].[parcel]
    JOIN [urbansim].[ref].[vi_parcel_geo_names]
    ON vi_parcel_geo_names.parcel_id = parcel.parcel_id
    LEFT JOIN  [urbansim].[urbansim].[scheduled_development_parcel]
    ON scheduled_development_parcel.parcel_id = parcel.parcel_id
    LEFT JOIN   (SELECT * FROM [urbansim].[urbansim].[additional_capacity]
                WHERE additional_capacity.version_id = %s) as additional_cap
    ON additional_cap.parcel_id = parcel.parcel_id
    GROUP BY jcpa,jcpa_name'''
capacity_sql = capacity_sql % versions['additional_capacity_version']
allcapacity = pd.read_sql(capacity_sql,mssql_engine)
#531591.0

In [ ]:
allcapacity.capacity.sum()

In [ ]:
allcapacity.head(2)

In [ ]:
len(allcapacity)

# target housing units

In [ ]:
hu_sql = '''
   SELECT  [version_id]
      ,[yr]
      ,[housing_units_add]
  FROM [urbansim].[urbansim].[urbansim_target_housing_units]
  where version_id =%s'''
hu_sql = hu_sql % versions['target_housing_units_version']
hu = pd.read_sql(hu_sql,mssql_engine)

In [ ]:
hu.head(2)

In [ ]:
hu.housing_units_add.sum()

## remaining capacity

In [ ]:
allcapacity.capacity.sum() - hu.housing_units_add.sum()

## subregional controls

In [ ]:
subregion_sql = '''
  SELECT  [subregional_crtl_id]
      ,[yr]
      ,[geo_id] as jcpa
      ,[control]
      ,[scenario_desc]
  FROM [urbansim].[urbansim].[urbansim_lite_subreg_control]
  where subregional_crtl_id = %s
  order by yr,geo_id'''
subregion_sql = subregion_sql % versions['subregional_ctrl_id']
subregion = pd.read_sql(subregion_sql,mssql_engine)

# units by year

In [ ]:
targets = pd.merge(hu,subregion,on='yr',how='left')

In [ ]:
targets['units'] = targets['control'] * targets['housing_units_add']

In [ ]:
targets.head(2)

# cumulative sum of units

In [ ]:
targets['cumsum_of_units'] = (targets.groupby(by=['jcpa'])['units'].cumsum()).round(2)

In [ ]:
target_and_capacity = pd.merge(targets[['yr','jcpa','units','cumsum_of_units']],allcapacity,on='jcpa',how='left')

In [ ]:
#target_and_capacity.loc[target_and_capacity.jcpa==4]

## compare targets and capacity

In [ ]:
t2050 = target_and_capacity.loc[target_and_capacity.yr==2050].copy()

In [ ]:
t2050['capacity_minus_target'] = (t2050['capacity']- t2050['cumsum_of_units']).round(2)

# do targets exceed capacity?

In [ ]:
not_enough_capacity = t2050.loc[t2050.capacity_minus_target<0].copy()

In [ ]:
if len(not_enough_capacity) > 0:
    print('\n\nTargets exceed capacity for the following jur or cpa:\n\n')
    print(not_enough_capacity)
else: print('Targets do not exceed capacity.')

In [ ]:
if len(not_enough_capacity) > 0: 
    plot_not_enough = target_and_capacity.loc[target_and_capacity.jcpa.isin(not_enough_capacity.jcpa.tolist())]

In [ ]:
if len(not_enough_capacity) > 0: 
    for f in not_enough_capacity['jcpa_name'].tolist(): 
         plot_not_enough[plot_not_enough['jcpa_name']==f][['yr','capacity','cumsum_of_units']].plot(x='yr',title=f)